In [2]:
import pandas as pd
import numpy as np
import math
import ast 

import jax
import jax.numpy as jnp
from jax import random, grad
from jax.nn import relu, sigmoid
from functools import partial

import matplotlib.pyplot as plt

import pytz
import os
import random
import sys
import pickle
from datetime import datetime

In [3]:
def has_carry_in_binary(a, b):
    carry = 0
    while a > 0 or b > 0:  # Mientras queden bits en a o b
        bit_a = a & 1  # Extrae el bit menos significativo de a
        bit_b = b & 1  # Extrae el bit menos significativo de b
        # Verificar si hay llevada en el bit actual
        if (bit_a + bit_b + carry) >= 2:
            return True
        # Actualizar carry para el próximo bit
        carry = (bit_a + bit_b + carry) // 2
        # Avanzar al siguiente bit
        a >>= 1
        b >>= 1
    return False

def has_carry_in_decimal(a, b):
    carry = 0
    while a > 0 or b > 0:  # Mientras queden dígitos en a o b
        digit_a = a % 10  # Extrae el dígito menos significativo de a
        digit_b = b % 10  # Extrae el dígito menos significativo de b
        # Verificar si hay llevada en el dígito actual
        if (digit_a + digit_b + carry) >= 10:
            return True
        # Actualizar carry para el próximo dígito
        carry = (digit_a + digit_b + carry) // 10
        # Avanzar al siguiente dígito
        a //= 10
        b //= 10
    return False

In [4]:
# Generar todas las combinaciones de números entre 0 y 100 con llevadas
real_test_dataset_with_carry_over = [
    (a, b) for a in range(128) for b in range(128) if has_carry_in_binary(a, b)
]

# Guardar las combinaciones en un archivo
file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/real_test_dataset_with_carry_over.txt'
with open(file_path, 'w') as file:
    file.write(str(real_test_dataset_with_carry_over))

file_path

'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/real_test_dataset_with_carry_over.txt'

In [ ]:
# Generar todas las combinaciones de números entre 0 y 100 con llevadas
combinations_with_carry = [
    (a, b) for a in range(100) for b in range(100) if has_carry_in_binary(a, b)
]

# Guardar las combinaciones en un archivo
file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/combinations_with_carry_over.txt'
with open(file_path, 'w') as file:
    file.write(str(combinations_with_carry))

file_path

In [ ]:
# Generar todas las combinaciones de números entre 0 y 99 con llevadas
combinations_with_carry_decimal = [
    (a, b) for a in range(100) for b in range(100) if has_carry_in_decimal(a, b)
]

# Guardar las combinaciones en un archivo
file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/combinations_with_carry_over_decimal.txt'
with open(file_path, 'w') as file:
    file.write(str(combinations_with_carry_decimal))

file_path

In [5]:
# Generar todas las combinaciones de números entre 0 y 127 con llevadas
real_test_dataset_with_carry_over_decimal = [
    (a, b) for a in range(128) for b in range(128) if has_carry_in_decimal(a, b)
]

# Guardar las combinaciones en un archivo
file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/real_test_dataset_with_carry_over_decimal.txt'
with open(file_path, 'w') as file:
    file.write(str(real_test_dataset_with_carry_over_decimal))

file_path

'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/real_test_dataset_with_carry_over_decimal.txt'

In [5]:
# Generar todas las combinaciones posibles entre 0 y 100
all_combinations = [(a, b) for a in range(100) for b in range(100)]

# Separar combinaciones con llevadas y sin llevadas
with_carry = [pair for pair in all_combinations if pair in combinations_with_carry]
without_carry = [pair for pair in all_combinations if pair not in combinations_with_carry]

# Calcular el 80% de las combinaciones (entrenamiento)
train_size = int(0.8 * len(all_combinations))

# Calcular el 80% de cada conjunto (con llevadas y sin llevadas)
with_carry_train_size = int(train_size * (len(with_carry) / len(all_combinations)))
without_carry_train_size = train_size - with_carry_train_size

# Seleccionar combinaciones para entrenamiento
train_with_carry = random.sample(with_carry, with_carry_train_size)
train_without_carry = random.sample(without_carry, without_carry_train_size)

# Combinar ambas listas para entrenamiento
train_couples = train_with_carry + train_without_carry

# Mezclar las parejas para tener una distribución aleatoria
random.shuffle(train_couples)

# Guardar las combinaciones de entrenamiento en un archivo
train_file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/train_couples.txt'
with open(train_file_path, 'w') as file:
    file.write(str(train_couples))

train_file_path

'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/train_couples.txt'

In [2]:
# Definir el rango máximo para los valores de 7 y 8 bits
max_7_bits = 2**7 - 1  # Valor máximo para 7 bits: 127
max_8_bits = 2**8 - 1  # Valor máximo para 8 bits: 255

# Crear una lista para almacenar las parejas que cumplen la condición
dataset = []

# Iterar sobre todos los valores posibles para a_i y b_i en el rango de 7 bits
for a in range(max_7_bits + 1):
    for b in range(max_7_bits + 1):
        if a + b <= max_8_bits:  # Comprobar si la suma cabe en 8 bits
            dataset.append((a, b))

# Guardar los datos en un archivo con el formato solicitado
file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/real_test_dataset.txt'
with open(file_path, "w") as file:
    file.write(str(dataset))

file_path

'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/real_test_dataset.txt'

In [4]:
# Crear una lista para almacenar las parejas que cumplen la condición
all_combinations = [(a, b) for a in range(100) for b in range(100)]

# Guardar los datos en un archivo con el formato solicitado
file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/test_dataset.txt'
with open(file_path, "w") as file:
    file.write(str(all_combinations))

file_path

'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/test_dataset.txt'

In [3]:
# Generar combinaciones donde al menos uno de los números está entre 0 y 9
combinations_with_0_to_9 = [
    (a, b) for a in range(100) for b in range(100) if (0 <= a <= 9 or 0 <= b <= 9)
]

# Generar combinaciones donde ambos números están entre 10 y 19
#combinations_10_to_19 = [
#    (a, b) for a in range(10, 20) for b in range(10, 20)
#]

# Combinar ambas listas
all_combinations = combinations_with_0_to_9 #+ combinations_10_to_19

# Guardar las combinaciones en un archivo
file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/combinations_small_problem_size.txt'
with open(file_path, 'w') as file:
    file.write(str(all_combinations))

file_path

'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/combinations_small_problem_size.txt'

In [6]:
# Generar combinaciones donde al menos uno de los números está entre 0 y 9
combinations_with_0_to_9 = [
    (a, b) for a in range(128) for b in range(128) if (0 <= a <= 9 or 0 <= b <= 9)
]

# Generar combinaciones donde ambos números están entre 10 y 19
combinations_10_to_19 = [
    (a, b) for a in range(10, 20) for b in range(10, 20)
]

# Combinar ambas listas
all_combinations = combinations_with_0_to_9 + combinations_10_to_19

# Guardar las combinaciones en un archivo
file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/real_test_dataset_small_problem_size.txt'
with open(file_path, 'w') as file:
    file.write(str(all_combinations))

file_path

'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/real_test_dataset_small_problem_size.txt'

In [2]:
# Generar combinaciones donde al menos uno de los números está entre 0 y 9
combinations_with_0_to_63 = [
    (a, b) for a in range(100) for b in range(100) if (0 <= a <= 63 or 0 <= b <= 63)
]

# Generar combinaciones donde ambos números están entre 10 y 19
#combinations_10_to_19 = [
#    (a, b) for a in range(10, 20) for b in range(10, 20)
#]

# Combinar ambas listas
all_combinations = combinations_with_0_to_63 #+ combinations_10_to_19

# Guardar las combinaciones en un archivo
file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/combinations_small_problem_size_binary.txt'
with open(file_path, 'w') as file:
    file.write(str(all_combinations))

file_path

'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/combinations_small_problem_size_binary.txt'

In [7]:
# Generar combinaciones donde al menos uno de los números está entre 0 y 9
combinations_with_0_to_63 = [
    (a, b) for a in range(128) for b in range(128) if (0 <= a <= 63 or 0 <= b <= 63)
]

# Generar combinaciones donde ambos números están entre 10 y 19
#combinations_10_to_19 = [
#    (a, b) for a in range(10, 20) for b in range(10, 20)
#]

# Combinar ambas listas
all_combinations = combinations_with_0_to_63 #+ combinations_10_to_19

# Guardar las combinaciones en un archivo
file_path = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/real_test_dataset_small_problem_size_binary.txt'
with open(file_path, 'w') as file:
    file.write(str(all_combinations))

file_path

'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition/real_test_dataset_small_problem_size_binary.txt'